# 10. [Pièges courants et pratiques recommandées](https://scikit-learn.org/stable/common_pitfalls.html#randomness)

Le but de ce chapitre est d'illustrer certains pièges et anti-modèles courants qui se produisent lors de l'utilisation de scikit-learn. Il fournit des exemples de ce qu'il **ne faut pas** faire, ainsi qu'un exemple correct correspondant.

# 10.1. Prétraitement incohérent

scikit-learn fournit une bibliothèque de [6. transformations d'ensembles de données](https://scikit-learn.org/stable/data_transforms.html#data-transforms), qui peuvent nettoyer (voir [6.3. Prétraitement des données](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)), réduire (voir [6.5. Réduction de dimensionnalité non supervisée](https://scikit-learn.org/stable/modules/unsupervised_reduction.html#data-reduction)), étendre (voir [6.7. Approximation du noyau](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation)) ou générer (voir [6.2. Extraction de caractéristiques](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-extraction)) des représentations de caractéristiques. Si ces transformations de données sont utilisées lors de l'entraînement' d'un modèle, elles doivent également être utilisées sur les ensembles de données suivants, qu'il s'agisse de données de test ou de données dans un système de production. Sinon, l'espace des caractéristiques changera et le modèle ne pourra pas fonctionner efficacement.

Pour l'exemple suivant, créons un jeu de données synthétique avec une seule caractéristique :

In [2]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

**Mauvais**

L'ensemble de données d'entraînement est mis à l'échelle, mais pas l'ensemble de données de test, de sorte que les performances du modèle sur l'ensemble de données de test sont moins bonnes que prévu :

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
model = LinearRegression().fit(X_train_transformed, y_train)
mean_squared_error(y_test, model.predict(X_test))

62.80867119249539


**Correct**

Au lieu de passer le `X_test` non transformé à `predict`, nous devrions transformer les données de test, de la même manière que nous avons transformé les données d'apprentissage :

In [4]:
X_test_transformed = scaler.transform(X_test)
mean_squared_error(y_test, model.predict(X_test_transformed))

0.902797546636954

Alternativement, nous recommandons d'utiliser un [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline), qui facilite l'enchaînement des transformations avec les estimateurs, et réduit la possibilité d'oublier une transformation :

In [5]:
from sklearn.pipeline import make_pipeline #, Pipeline

model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)
# Pipeline(steps=[('standardscaler', StandardScaler()),
#               ('linearregression', LinearRegression())])
mean_squared_error(y_test, model.predict(X_test))

0.902797546636954

Les pipelines permettent également d'éviter un autre écueil courant : la fuite des données de test dans les données d'apprentissage.

# 10.2. Fuite de données

Une fuite de données se produit lorsque des informations qui ne seraient pas disponibles au moment de la prédiction sont utilisées lors de la construction du modèle. Cela se traduit par des estimations de performances trop optimistes, par exemple à partir de la [3.1. validation croisée](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation), et donc des performances plus faibles lorsque le modèle est utilisé sur des données réellement nouvelles, par exemple pendant la production.

Une cause courante est de ne pas séparer les sous-ensembles de données de test et d'entraînement. Les données de test ne doivent jamais être utilisées pour faire des choix sur le modèle. **La règle générale est de ne jamais appeler** `fit` **sur les données de test**. Bien que cela puisse sembler évident, il est facile de passer à côté dans certains cas, par exemple lors de l'application de certaines étapes de prétraitement.

Bien que les sous-ensembles de données d'apprentissage et de test doivent recevoir la même transformation de prétraitement (comme décrit dans la section précédente), il est important que ces transformations ne soient apprises qu'à partir des données d'apprentissage. Par exemple, si vous avez une étape de normalisation dans laquelle vous divisez par la valeur moyenne, la moyenne doit être la moyenne du sous-ensemble d'nentraînement, et non la moyenne de toutes les données. Si le sous-ensemble de test est inclus dans le calcul de la moyenne, les informations du sous-ensemble de test influencent le modèle.

Un exemple de fuite de données lors du prétraitement est détaillé ci-dessous.

## 10.2.1. Fuite de données lors du prétraitement

**NB** - Nous choisissons ici d'illustrer la fuite de données avec une étape de sélection des caractéristiques. Ce risque de fuite est cependant pertinent avec presque toutes les transformations dans scikit-learn, y compris (mais sans s'y limiter) [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler), [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) et [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA).

Un certain nombre de [1.13. fonctions de sélection de caractéristiques](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) sont disponibles dans scikit-learn. Elles peuvent aider à supprimer les caractéristiques non pertinentes, redondantes et bruyantes, ainsi qu'à améliorer le temps de construction et les performances de votre modèle. Comme pour tout autre type de prétraitement, la sélection des caractéristiques ne doit utiliser que les données d'apprentissage. L'inclusion des données de test dans la sélection des caractéristiques biaisera votre modèle de manière optimiste.

Pour le démontrer, nous allons créer ce problème de classification binaire avec 10 000 caractéristiques générées aléatoirement :

In [6]:
import numpy as np
n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

**Mauvais**

L'utilisation de toutes les données pour effectuer la sélection des caractéristiques donne un score de précision bien supérieur au hasard, même si nos cibles sont complètement aléatoires. Ce caractère aléatoire signifie que nos `X` et `y` sont indépendants et nous nous attendons donc à ce que la précision soit d'environ 0,5. Cependant, puisque l'étape de sélection des caractéristiques "voit" les données de test, le modèle a un avantage injuste. Dans l'exemple incorrect ci-dessous, nous utilisons d'abord toutes les données pour la sélection des caractéristiques, puis nous divisons les données en sous-ensembles d'apprentissage et de test pour l'ajustement du modèle. Le résultat est un score de précision beaucoup plus élevé que prévu :

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Incorrect preprocessing: the entire data is transformed
X_selected = SelectKBest(k=25).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, random_state=42)
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train, y_train)
# GradientBoostingClassifier(random_state=1)

y_pred = gbc.predict(X_test)
accuracy_score(y_test, y_pred)

0.76

**Correct**

Pour éviter les fuites de données, il est recommandé de diviser d'abord vos données en sous-ensembles d'entraînement et de test. La sélection dde caractéristiques peut ensuite être entraînée en utilisant uniquement l'ensemble de données d'apprentissage. Notez que chaque fois que nous utilisons `fit` ou `fit_transform`, nous n'utilisons que le jeu de données d'apprentissage. Le score est à présent ce à quoi nous nous attendions pour les données, proche du hasard :

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
select = SelectKBest(k=25)
X_train_selected = select.fit_transform(X_train, y_train)

gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)
# GradientBoostingClassifier(random_state=1)

X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.46

Là encore, nous vous recommandons d'utiliser un [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) pour enchaîner la sélection de caractéristiques et les estimateurs de modèle. Le pipeline garantit que seules les données d'entraînement sont utilisées lors de l'ajustement et que les données de test sont utilisées uniquement pour calculer le score de précision :

In [9]:
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
pipeline = make_pipeline(SelectKBest(k=25),
    GradientBoostingClassifier(random_state=1))
pipeline.fit(X_train, y_train)
# Pipeline(steps=[('selectkbest', SelectKBest(k=25)),
#                ('gradientboostingclassifier',
#                GradientBoostingClassifier(random_state=1))])

y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.46

Le pipeline peut également être introduit dans une fonction de validation croisée telle que [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score). Là encore, le pipeline garantit que le sous-ensemble de données et la méthode d'estimation corrects sont utilisés lors de l'ajustement et de la prédiction :

In [10]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y)
print(f"Mean accuracy: {scores.mean():.2f}+/-{scores.std():.2f}")

Mean accuracy: 0.45+/-0.07


## 10.2.2. Comment éviter les fuites de données

Voici quelques conseils pour éviter les fuites de données :

* Commencez toujours par diviser les données en sous-ensembles d'entraînement et de test, en particulier avant toute étape de prétraitement.

* N'incluez jamais de données de test lorsque vous utilisez les méthodes `fit` et `fit_transform`. L'utilisation de toutes les données, par exemple `fit(X)`, peut entraîner des scores trop optimistes.

* À l'inverse, la méthode de transformation doit être utilisée à la fois sur les sous-ensembles d'entraînement et de test, car le même prétraitement doit être appliqué à toutes les données. Ceci peut être réalisé en utilisant `fit_transform` sur le sous-ensemble d'entraînement et `transform` sur le sous-ensemble de test.

* Le [6.1.1. pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline) scikit-learn est un excellent moyen d'éviter les fuites de données car il garantit que la méthode appropriée est exécutée sur le sous-ensemble de données approprié. Le pipeline est idéal pour une utilisation dans les fonctions de validation croisée et de réglage des hyperparamètres.

# 10.3. Contrôler l'aléatoire